In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io

In [3]:
path_to_zip = keras.utils.get_file('spa-eng.zip', origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)

In [4]:
path_to_zip

'C:\\Users\\Hacker\\.keras\\datasets\\spa-eng.zip'

In [5]:
path_to_file = os.path.join(os.path.dirname(path_to_zip), 'spa-eng', 'spa.txt')

In [6]:
path_to_file

'C:\\Users\\Hacker\\.keras\\datasets\\spa-eng\\spa.txt'

In [7]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [8]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w

In [9]:
en_sentence = "I have some friends to help."
preprocess_sentence(en_sentence)

'<start> i have some friends to help . <end>'

In [10]:
sp_sentence = "Aún no he dicho nada."
preprocess_sentence(sp_sentence)

'<start> aun no he dicho nada . <end>'

In [11]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    return zip(*word_pairs)

In [12]:
create_dataset(path_to_file, 3)

In [13]:
en, sp = create_dataset(path_to_file, None)

In [14]:
len(en)

118964

In [15]:
def max_lenght(tensor):
    return max(len(t) for t in tensor)

In [16]:
def tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [17]:
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [18]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)

In [19]:
input_tensor

array([[  1,  97,   3, ...,   0,   0,   0],
       [  1, 179,   3, ...,   0,   0,   0],
       [  1, 466,   3, ...,   0,   0,   0],
       ...,
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0]])

In [20]:
max_length_targ, max_length_inp = max_lenght(input_tensor), max_lenght(target_tensor)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [22]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [23]:
input_tensor[0]

array([ 1, 97,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [24]:
convert(input_lang_tokenizer, input_tensor[0])

1  ....  <start>
97  ....  ve
3  ....  .
2  ....  <end>


In [25]:
target_lang_tokenizer.index_word

{1: '<start>',
 2: '<end>',
 3: '.',
 4: 'i',
 5: 'you',
 6: '?',
 7: 'tom',
 8: 'is',
 9: 'it',
 10: 's',
 11: 'a',
 12: 'he',
 13: 't',
 14: 'the',
 15: 'we',
 16: 'm',
 17: 'me',
 18: 're',
 19: 'that',
 20: 'this',
 21: 'to',
 22: 'do',
 23: 'are',
 24: 'can',
 25: 'my',
 26: 'they',
 27: 'was',
 28: 'she',
 29: 'don',
 30: 'have',
 31: 'your',
 32: 'go',
 33: 'what',
 34: 'in',
 35: 'not',
 36: 'll',
 37: 'like',
 38: 'here',
 39: 'on',
 40: 'him',
 41: 'let',
 42: 'be',
 43: 'did',
 44: 'know',
 45: 'come',
 46: 'up',
 47: 'am',
 48: 'want',
 49: 'how',
 50: ',',
 51: 'mary',
 52: 'get',
 53: 'who',
 54: 'very',
 55: 'now',
 56: 'need',
 57: 'has',
 58: 'please',
 59: 'no',
 60: 'there',
 61: 'help',
 62: 'her',
 63: 'love',
 64: 'at',
 65: 'see',
 66: 'just',
 67: 'out',
 68: 'his',
 69: 've',
 70: 'got',
 71: 'for',
 72: 'where',
 73: 'look',
 74: 'stop',
 75: 'one',
 76: 'us',
 77: 'good',
 78: 'car',
 79: 'too',
 80: 'so',
 81: 'all',
 82: 'why',
 83: 'will',
 84: 'an',
 85: 

In [26]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

In [27]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [28]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [29]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [30]:
encoder

In [31]:
simple_hidden = encoder.initialize_hidden_state()
simple_hidden

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [32]:
example_input_batch, example_output_batch = next(iter(dataset))

In [33]:
encoder(example_input_batch, simple_hidden)

(<tf.Tensor: shape=(64, 16, 1024), dtype=float32, numpy=
 array([[[-3.81078478e-03, -6.88044401e-03, -4.75821132e-03, ...,
           2.65271030e-03,  6.67464873e-03, -2.40614102e-03],
         [-1.60770165e-03, -4.46899794e-04, -6.26616040e-03, ...,
          -1.01432868e-03, -6.45269174e-05, -3.36275832e-03],
         [-6.00970117e-03, -9.80717875e-03, -7.96431582e-03, ...,
           2.46365787e-03, -1.56786220e-04,  4.06825356e-03],
         ...,
         [-1.41691770e-02, -7.67568592e-03,  1.21137165e-02, ...,
           7.87413213e-04,  1.88760902e-03,  6.56944234e-03],
         [-1.43058458e-02, -7.98291527e-03,  1.25614312e-02, ...,
           4.72557207e-04,  2.13799253e-03,  6.56569190e-03],
         [-1.43625867e-02, -8.19821469e-03,  1.28151607e-02, ...,
           2.35170693e-04,  2.27858033e-03,  6.58854609e-03]],
 
        [[-3.81078478e-03, -6.88044401e-03, -4.75821132e-03, ...,
           2.65271030e-03,  6.67464873e-03, -2.40614102e-03],
         [-1.15837893e-02, -1.

In [34]:
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [35]:
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [36]:
attention_layer = Attention(10)
attention_layer(simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[-0.00637142, -0.0011614 ,  0.00246351, ...,  0.00320543,
          0.00173788,  0.0025068 ],
        [-0.00713754, -0.00466628,  0.00087075, ...,  0.00310231,
         -0.00075256,  0.00410261],
        [-0.00434387, -0.00376534,  0.00662781, ...,  0.00301996,
          0.00232902,  0.00440815],
        ...,
        [-0.00694919, -0.00345998,  0.00228049, ..., -0.00040365,
         -0.00039935,  0.00284764],
        [-0.0085916 , -0.00443213,  0.00328777, ...,  0.00262704,
          0.00125907,  0.00528511],
        [-0.00681534, -0.0058036 ,  0.00294724, ...,  0.00479816,
          0.00115968,  0.00433635]], dtype=float32)>,
 <tf.Tensor: shape=(64, 16, 1), dtype=float32, numpy=
 array([[[0.06260864],
         [0.06200516],
         [0.06119423],
         ...,
         [0.06337715],
         [0.0633753 ],
         [0.06337006]],
 
        [[0.06236051],
         [0.06222363],
         [0.06182159],
         ...,
         [0.

In [37]:
attention_result, attention_weights = attention_layer(simple_hidden, simple_output)

In [38]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [39]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [40]:
decoder

In [41]:
decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 3727), dtype=float32, numpy=
 array([[-5.9663528e-04, -9.2281611e-04,  3.1096116e-03, ...,
         -4.0878035e-04,  1.3587471e-03,  1.0131508e-03],
        [-1.0738734e-03, -1.0389721e-03,  2.9886661e-03, ...,
         -3.3308711e-04,  1.5985999e-03,  6.8748649e-04],
        [-1.0771633e-03, -8.5626956e-04,  3.0706492e-03, ...,
         -5.6035235e-05,  1.7186156e-03,  1.8436906e-03],
        ...,
        [-1.0118932e-03, -1.1757703e-03,  2.6470064e-03, ...,
          2.2588432e-04,  1.1720227e-03,  1.2730743e-03],
        [-1.6822740e-03, -1.3195685e-03,  2.7264822e-03, ...,
         -2.7071417e-04,  1.8500630e-03,  1.7058863e-03],
        [-1.0642146e-03, -7.3215301e-04,  3.0915393e-03, ...,
         -2.2158367e-04,  2.1131495e-03,  8.6248538e-04]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 0.00581166,  0.00582146,  0.00228046, ..., -0.00480631,
         -0.00498638,  0.00283025],
        [ 0.00568624,  0.00521999,  0.001

In [42]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [43]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [44]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [45]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [46]:
# EPOCH = 20
# for epoch in range(EPOCH):
#     enc_hidden = encoder.initialize_hidden_state()
#     total_loss = 0
#     for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
#         batch_loss = train_step(inp, targ, enc_hidden)
#         total_loss += batch_loss
#         print('Epoch: ', epoch)
#         print('Loss: ', batch_loss.numpy())
#     checkpoint.save(file_prefix='chckpnts/test1')

In [49]:
checkpoint.restore(tf.train.latest_checkpoint('chckpnts'))

In [56]:
def Evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequence([inputs], maxlen=max_len_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)
    for t in range(max_len_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [57]:
evaluate('hace mucho frio aqui.')

('it s very cold here . <end> ', '<start> hace mucho frio aqui . <end>')